In [4]:
!pip install selenium


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install undetected_chromedriver


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from qdrant_client import QdrantClient

# Kiểm tra kết nối tới Qdrant
client = QdrantClient(host='localhost', port=6333)  # Đảm bảo đúng cổng và host
try:
    response = client.get_collection("your_collection_name")
    print(response)
except Exception as e:
    print(f"Error: {e}")


Error: [WinError 10061] No connection could be made because the target machine actively refused it


In [9]:
import shutil
import os
import tempfile
import pandas as pd
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import logging
from fake_useragent import UserAgent
import random

# Set up logging configuration
logging.basicConfig(
    filename='province_search.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# List of 63 provinces in Vietnam
provinces = ["Hồ Chí Minh"]


def init_selenium_driver():
    ua = UserAgent()
    user_agent = ua.random  # Fixed user agent per session
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--disable-features=MetricsReporting")
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.managed_default_content_settings.stylesheets": 2,
        "profile.managed_default_content_settings.javascript": 1
    }
    options.add_experimental_option("prefs", prefs)
    
    # Create a temporary directory for user data
    temp_user_data_dir = tempfile.mkdtemp()
    options.add_argument(f'--user-data-dir={temp_user_data_dir}')
    
    # Set the `version_main` parameter to match your Chrome version
    driver = uc.Chrome(options=options, version_main=130)
    return driver, temp_user_data_dir


# Function to search on Bing and retrieve the first 5 links
def search_province_links(driver, province_name):
    search_query = f"cẩm nang du lịch {province_name}"
    search_url = f"https://www.bing.com/search?q={search_query}"
    
    logging.info(f"Searching on Bing: {search_url}")
    driver.get(search_url)
    
    # Wait for search results to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.b_algo')))
    
    # Extract first 5 URLs
    links = []
    try:
        search_results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo')[:10]
        for result in search_results:
            link_element = result.find_element(By.TAG_NAME, 'a')
            links.append(link_element.get_attribute('href'))
    except Exception as e:
        logging.error(f"Error retrieving links for {province_name}: {e}")
    
    return links

# Function to write the results to CSV
def write_results_to_csv(province_name, links, csv_file):
    links_str = "; ".join(links) if links else "No links found"
    df = pd.DataFrame({'Province': [province_name], 'Websites': [links_str]})
    df.to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)

# Main function to process each province one by one
def process_provinces(provinces, csv_output_file):
    driver, temp_user_data_dir = init_selenium_driver()
    try:
        for province in provinces:
            links = search_province_links(driver, province)
            write_results_to_csv(province, links, csv_output_file)
            time.sleep(random.uniform(5, 8))  # Adjust delay to reduce risk of rate limits
    finally:
        driver.quit()  # Close the driver
        shutil.rmtree(temp_user_data_dir)  # Clean up the temporary directory

# Usage
csv_output_file = 'province_websites.csv'
process_provinces(provinces, csv_output_file)


PermissionError: [WinError 5] Access is denied: 'C:\\Users\\Dell\\AppData\\Local\\Temp\\tmp782p9jvc\\BrowserMetrics\\BrowserMetrics-67341204-1860.pma'

In [11]:
import shutil
import os
import tempfile
import pandas as pd
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import logging
from fake_useragent import UserAgent
import random
import requests
from bs4 import BeautifulSoup
import re

# Set up logging configuration
logging.basicConfig(
    filename='province_search.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# List of 63 provinces in Vietnam
provinces = ["Hồ Chí Minh"]

def init_selenium_driver():
    ua = UserAgent()
    user_agent = ua.random  # Fixed user agent per session
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--disable-features=MetricsReporting")
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.managed_default_content_settings.stylesheets": 2,
        "profile.managed_default_content_settings.javascript": 1
    }
    options.add_experimental_option("prefs", prefs)
    
    # Create a temporary directory for user data
    temp_user_data_dir = tempfile.mkdtemp()
    options.add_argument(f'--user-data-dir={temp_user_data_dir}')
    
    # Set the `version_main` parameter to match your Chrome version
    driver = uc.Chrome(options=options, version_main=130)
    return driver, temp_user_data_dir

# Function to search on Bing and retrieve the first 5 links
def search_province_links(driver, province_name):
    search_query = f"cẩm nang du lịch {province_name}"
    search_url = f"https://www.bing.com/search?q={search_query}"
    
    logging.info(f"Searching on Bing: {search_url}")
    driver.get(search_url)
    
    # Wait for search results to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.b_algo')))
    
    # Extract first 5 URLs
    links = []
    try:
        search_results = driver.find_elements(By.CSS_SELECTOR, 'li.b_algo')[:5]
        for result in search_results:
            link_element = result.find_element(By.TAG_NAME, 'a')
            links.append(link_element.get_attribute('href'))
    except Exception as e:
        logging.error(f"Error retrieving links for {province_name}: {e}")
    
    return links

# Function to retrieve and clean content from a URL
def get_info_header(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
        }
        # Get the HTML content from the URL with headers
        response = requests.get(url, headers=headers)
        html_text = response.text

        # Parse the HTML content using lxml parser
        soup = BeautifulSoup(html_text, 'lxml')

        # Find all <p> tags and return their content as a string
        content = ''
        for p in soup.find_all('p'):
            content += p.get_text() + '\n'
            
        # Clean the content by removing excessive newlines and leading/trailing spaces
        content = re.sub(r'\n+', '\n', content).strip()
        
        return content
    except Exception as e:
        logging.error(f"Error while processing {url}: {e}")
        return None

# Function to write the results to CSV
def write_results_to_csv(province_name, link, content, csv_file):
    df = pd.DataFrame({'Province': [province_name], 'Link': [link], 'Content': [content]})
    df.to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)

# Main function to process each province one by one
def process_provinces(provinces, csv_output_file):
    driver, temp_user_data_dir = init_selenium_driver()
    try:
        for province in provinces:
            links = search_province_links(driver, province)
            for link in links:
                content = get_info_header(link)
                write_results_to_csv(province, link, content, csv_output_file)
                time.sleep(random.uniform(5, 8))  # Adjust delay to reduce risk of rate limits
    finally:
        driver.quit()  # Close the driver
        shutil.rmtree(temp_user_data_dir)  # Clean up the temporary directory

# Usage
csv_output_file = 'province_websites_content.csv'
process_provinces(provinces, csv_output_file)


PermissionError: [WinError 5] Access is denied: 'C:\\Users\\Dell\\AppData\\Local\\Temp\\tmplhdg1e96\\BrowserMetrics\\BrowserMetrics-6734143C-438C.pma'

In [15]:
df = pd.read_csv('province_websites_content.csv')
df

,Province,Link,Content
0,Hồ Chí Minh,https://vnexpress.net/cam-nang-du-lich-tp-hcm-...,"TP HCM là nơi hội tụ nhiều nền văn hóa, với cá..."
1,Hồ Chí Minh,https://dulich24.com.vn/du-lich-tphcm/tp-ho-ch...,Du lịch đến với Sài Gòn – TP.HCM hơn 300 năm t...
2,Hồ Chí Minh,https://www.visithcmc.vn/news/top-20-places-yo...,Văn Hóa & Lịch Sử\nVới sự kết hợp giữa vẻ đẹp ...
3,Hồ Chí Minh,https://www.agoda.com/vi-vn/travel-guides/viet...,"Chào mừng bạn đến với Thành phố Hồ Chí Minh, m..."


In [21]:
# Đọc file CSV, dùng quotechar để xử lý các trích dẫn bên trong
df = pd.read_csv("province_websites_content_1.csv", quotechar='"')

# Sử dụng regex để loại bỏ tất cả văn bản nằm giữa dấu ngoặc kép "" trong cột Content
df['Content'] = df['Content'].str.replace(r'"[^"]*"', '', regex=True)
df['Content'] = df['Content'].str.replace(',', ';', regex=False)
# Xóa khoảng trắng dư thừa (nếu có) sau khi loại bỏ các đoạn văn bản nằm giữa dấu ngoặc kép
df['Content'] = df['Content'].str.strip()

# Kiểm tra lại kết quả

print(df['Content'])
df.to_csv("province_websites_content_1.csv",header=True, index=False)

0    TP HCM là nơi hội tụ nhiều nền văn hóa; với cá...
1    Du lịch đến với Sài Gòn – TP.HCM hơn 300 năm t...
2    Văn Hóa & Lịch Sử\nVới sự kết hợp giữa vẻ đẹp ...
3    Chào mừng bạn đến với Thành phố Hồ Chí Minh; m...
Name: Content, dtype: object


In [22]:
df = pd.read_csv("province_websites_content_1.csv", quotechar='"')
# Loại bỏ toàn bộ dòng trống hoặc khoảng trắng thừa trong cột Content
df['Content'] = df['Content'].str.replace(r'\s*\n\s*', ' ', regex=True).str.strip()
df.to_csv("province_websites_content_1.csv")